In [1]:
import os
from joblib import dump, load
import torch
import pandas as pd
import torch.nn as nn
from math import sqrt
from tqdm import tqdm

In [2]:
root_path = os.path.abspath(os.path.dirname(os.getcwd()))
train_path = os.path.join(root_path, 'data', 'train')
test_path = os.path.join(root_path, 'data', 'test')
processed_path = os.path.join(root_path, 'data', 'processed')

### Load `pred_y`

In [3]:
pred_y = load('predicted_y.joblib')

/usr/local/lib/python3.8/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [4]:
pred_y_df = pd.DataFrame(pred_y).astype('float')

In [5]:
print(pred_y)

tensor([[10.2043, 10.2502, 10.4696,  ..., 10.4181, 10.3028, 10.1739],
        [10.0367, 10.0803, 10.2987,  ..., 10.2425, 10.1338, 10.0046],
        [10.0366, 10.0802, 10.2986,  ..., 10.2424, 10.1338, 10.0045],
        ...,
        [10.3886, 10.4364, 10.6573,  ..., 10.6111, 10.4878, 10.3600],
        [10.1307, 10.1757, 10.3946,  ..., 10.3409, 10.2287, 10.0995],
        [10.1260, 10.1710, 10.3899,  ..., 10.3361, 10.2240, 10.0948]])


In [6]:
pred_y_df

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,10.204296,10.250236,10.469630,10.187499,10.242941,10.269687,10.320245,10.244794,10.199506,10.323218,...,10.312169,10.209535,10.281565,10.304939,10.166118,10.282874,10.255626,10.418071,10.302790,10.173872
1,10.036656,10.080258,10.298698,10.017245,10.072940,10.099568,10.151118,10.077601,10.032894,10.149748,...,10.141307,10.043319,10.112891,10.137672,10.000942,10.114313,10.085155,10.242466,10.133828,10.004580
2,10.036598,10.080204,10.298636,10.017194,10.072888,10.099515,10.151062,10.077546,10.032845,10.149697,...,10.141252,10.043261,10.112843,10.137616,10.000890,10.114258,10.085094,10.242416,10.133776,10.004522
3,10.405839,10.453745,10.674770,10.391726,10.446774,10.472551,10.522200,10.445010,10.398204,10.531754,...,10.516991,10.409320,10.483652,10.505136,10.362699,10.484732,10.459999,10.629051,10.505012,10.377305
4,10.156007,10.201399,10.420453,10.138502,10.194037,10.220907,10.271717,10.196768,10.151744,10.273242,...,10.263051,10.161670,10.233082,10.256907,10.118826,10.234454,10.206596,10.367492,10.254248,10.125095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,10.072140,10.116360,10.334950,10.053308,10.108991,10.135736,10.187055,10.113114,10.068336,10.186454,...,10.177571,10.078524,10.148695,10.173223,10.036137,10.150124,10.121292,10.279649,10.169674,10.040389
856,10.065352,10.109469,10.328018,10.046415,10.102108,10.128834,10.180184,10.106334,10.061571,10.179440,...,10.170646,10.071791,10.141857,10.166433,10.029422,10.143291,10.114389,10.272549,10.162827,10.033540
857,10.388643,10.436396,10.657279,10.374319,10.429394,10.455280,10.504998,10.427942,10.381293,10.513961,...,10.499529,10.392269,10.466420,10.488072,10.345967,10.467525,10.442572,10.611052,10.487782,10.359960
858,10.130667,10.175732,10.394629,10.112761,10.168358,10.195214,10.246172,10.171521,10.126584,10.247025,...,10.237239,10.136558,10.207604,10.231652,10.093889,10.209002,10.180832,10.340948,10.228712,10.099492


In [7]:
processed_global_run_data = pd.read_parquet(os.path.join(processed_path, 'processed_global_run_data.parquet'))
processed_global_incoming_run_data = pd.read_parquet(os.path.join(processed_path, 'processed_global_incoming_run_data.parquet'))
processed_global_metrology_data = pd.read_parquet(os.path.join(processed_path, 'processed_global_metrology_data.parquet'))

processed_test_run_data = pd.read_parquet(os.path.join(processed_path, 'processed_test_run_data.parquet'))
processed_test_incoming_run_data = pd.read_parquet(os.path.join(processed_path, 'processed_test_incoming_run_data.parquet'))

In [8]:
sorted_processed_test_run_data = processed_test_run_data.sort_values(
    by=['Tool ID', 'Run ID', 'Time Stamp'],
    ascending=[True, True, True]
)

sorted_processed_test_incoming_run_data = processed_test_incoming_run_data.sort_values(
    by=['Tool ID', 'Run ID', 'Time Stamp'],
    ascending=[True, True, True]
)

In [9]:
run_ids = []
for key, run in sorted_processed_test_run_data.groupby('Run ID'):
    run_ids.append(key)

In [10]:
pred_y_df['Run ID'] = run_ids
pred_y_df

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,Run ID
0,10.204296,10.250236,10.469630,10.187499,10.242941,10.269687,10.320245,10.244794,10.199506,10.323218,...,10.209535,10.281565,10.304939,10.166118,10.282874,10.255626,10.418071,10.302790,10.173872,3
1,10.036656,10.080258,10.298698,10.017245,10.072940,10.099568,10.151118,10.077601,10.032894,10.149748,...,10.043319,10.112891,10.137672,10.000942,10.114313,10.085155,10.242466,10.133828,10.004580,4
2,10.036598,10.080204,10.298636,10.017194,10.072888,10.099515,10.151062,10.077546,10.032845,10.149697,...,10.043261,10.112843,10.137616,10.000890,10.114258,10.085094,10.242416,10.133776,10.004522,8
3,10.405839,10.453745,10.674770,10.391726,10.446774,10.472551,10.522200,10.445010,10.398204,10.531754,...,10.409320,10.483652,10.505136,10.362699,10.484732,10.459999,10.629051,10.505012,10.377305,17
4,10.156007,10.201399,10.420453,10.138502,10.194037,10.220907,10.271717,10.196768,10.151744,10.273242,...,10.161670,10.233082,10.256907,10.118826,10.234454,10.206596,10.367492,10.254248,10.125095,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855,10.072140,10.116360,10.334950,10.053308,10.108991,10.135736,10.187055,10.113114,10.068336,10.186454,...,10.078524,10.148695,10.173223,10.036137,10.150124,10.121292,10.279649,10.169674,10.040389,4900
856,10.065352,10.109469,10.328018,10.046415,10.102108,10.128834,10.180184,10.106334,10.061571,10.179440,...,10.071791,10.141857,10.166433,10.029422,10.143291,10.114389,10.272549,10.162827,10.033540,4902
857,10.388643,10.436396,10.657279,10.374319,10.429394,10.455280,10.504998,10.427942,10.381293,10.513961,...,10.392269,10.466420,10.488072,10.345967,10.467525,10.442572,10.611052,10.487782,10.359960,4907
858,10.130667,10.175732,10.394629,10.112761,10.168358,10.195214,10.246172,10.171521,10.126584,10.247025,...,10.136558,10.207604,10.231652,10.093889,10.209002,10.180832,10.340948,10.228712,10.099492,4908


In [11]:
pred_y_df = pred_y_df.melt(
    id_vars='Run ID',
    var_name='Point Index',
    value_name="Measurement"
)

In [12]:
pred_y_df = pred_y_df.sort_values('Run ID')
pred_y_df

,Run ID,Point Index,Measurement
0,3,0,10.204296
34400,3,40,10.209535
2580,3,3,10.187499
39560,3,46,10.418071
7740,3,9,10.323218
...,...,...,...
12039,4909,13,10.169545
29239,4909,33,10.246445
12899,4909,14,10.062579
13759,4909,15,10.273788


In [13]:
global_run_data = pd.read_parquet(os.path.join(processed_path, 'global_run_data.parquet'))
global_incoming_run_data = pd.read_parquet(os.path.join(processed_path, 'global_incoming_run_data.parquet'))
global_metrology_data = pd.read_parquet(os.path.join(processed_path, 'global_metrology_data.parquet'))

test_run_data = pd.read_parquet(os.path.join(processed_path, 'rounded_test_run_data.parquet'))
test_incoming_run_data = pd.read_parquet(os.path.join(processed_path, 'rounded_test_incoming_run_data.parquet'))

In [14]:
tool_id_dict = {'8060e8e1-504a-5138-a9f0-e2770bd61ba1': 1,
                'ad28cd71-3811-543c-80a1-a440468a49d7': 2,
                'ac40636d-b491-5620-8d16-9aad56e5c4d3': 3,
                '08e35986-3a0a-5274-8990-4ad303be9d5f': 4,
                '2f2816e1-d294-5c4d-a5fa-3d804c46726a': 5,
                '8a1010f8-7e9e-52ba-8586-2eac7e68eaee': 6,
                '683e5405-4b93-5160-be90-d3d5b0d9287a': 7,
                'db66ded5-0f4f-5179-b8e7-4673304ddff1': 8,
                '5116a75f-0543-502f-8278-2b2496d337d2': 9,
                'c7f536aa-a2fc-54b4-9b98-9f01793c9b5d': 10,
                'a060c807-dbdb-5de3-9cd7-0c596234a3d7': 51,
                '1914935a-df63-54c2-9c47-da905244e631': 52,
                'b3439d8e-290b-59a4-96a5-53348288f6a9': 53,
                'd7291410-f34f-5cee-b3a1-0c60bb854423': 54,
                '1b314ddd-198a-5cd5-90ae-933b947d013d': 55,
                '30476bd4-f093-56c9-8cd1-23cf7f39ce5f': 56,
                'e4838ac1-3788-50bf-bbca-04cf339ec369': 57,
                '4738fd3d-8fc7-59fd-85a7-eec6efb4bfae': 58,
                'bb7e0385-0227-575a-9894-dacc8bf07f2a': 59,
                '3cb39167-0519-578a-846d-7132592965d3': 60}

run_id_set = set(test_run_data['Run ID'].unique()).union(set(global_run_data['Run ID'].unique()))
run_id_dict = {run_id: i + 1 for i, run_id in enumerate(list(run_id_set))}

### Two checks to confirm one-to-one mapping

In [15]:
violation_check_1 = global_metrology_data.groupby('Point Index')[['X', 'Y', 'X_index', 'Y_index']].nunique()
(violation_check_1 > 1).any().any()

False

In [16]:
violation_check_2 = global_metrology_data.groupby(['X', 'Y', 'X_index', 'Y_index'])['Point Index'].nunique()
(violation_check_2 > 1).any()

False

#### Join back the columns

In [17]:
# Define the columns that describe each unique point
point_columns = ['Point Index', 'X', 'Y', 'X_index', 'Y_index']

# Create a clean DataFrame where each Point Index appears only once
point_mapping_df = global_metrology_data[point_columns].drop_duplicates()

print("Created a unique mapping table with shape:", point_mapping_df.shape)
print(point_mapping_df.head())

Created a unique mapping table with shape: (49, 5)
   Point Index           X          Y  X_index  Y_index
0            3 -143.877551  -9.183673        1       23
1           48 -143.877551  27.551020        1       29
2           43 -137.755102  58.163265        2       34
3           20 -131.632653 -64.285714        3       14
4            8 -119.387755  88.775510        5       39


In [18]:
pred_y_df = pd.merge(
    left=pred_y_df,
    right=point_mapping_df,
    on='Point Index',
    how='left'  # Use a 'left' merge to ensure all rows from sorted_df are kept
)

# Display the final, combined DataFrame
print("\nSuccessfully merged the datasets.")
print("Final DataFrame shape:", pred_y_df.shape)
print(pred_y_df.head())


Successfully merged the datasets.
Final DataFrame shape: (42140, 7)
   Run ID Point Index  Measurement           X           Y  X_index  Y_index
0       3           0    10.204296    3.061224 -119.387755       25        5
1       3          40    10.209535   82.653061  -94.897959       38        9
2       3           3    10.187499 -143.877551   -9.183673        1       23
3       3          46    10.418071  131.632653    9.183673       46       26
4       3           9    10.323218  -58.163265  -33.673469       15       19


## Add back `Run Start Time` and `Run End Time`

* Metrology data share the same `Run Start Time` and `Run End Time` with the corresponding run data

In [ ]:
tool_1_run_data = global_run_data[global_run_data['Tool ID'] == '8060e8e1-504a-5138-a9f0-e2770bd61ba1']
tool_1_incoming_run_data = global_incoming_run_data[global_incoming_run_data['Tool ID'] == '8060e8e1-504a-5138-a9f0-e2770bd61ba1']
for key, run in tool_1_run_data.groupby('Run ID'):
    print(f"{key}: ")
    print(run['Run Start Time'].nunique())
    print(run['Run End Time'].nunique())
    incoming_run = tool_1_incoming_run_data[tool_1_incoming_run_data['Run ID'] == key]
    print(incoming_run['Run Start Time'].nunique())
    print(incoming_run['Run End Time'].nunique())
    print(run['Run Start Time'].iloc[0] == incoming_run['Run Start Time'].iloc[0])
    print(run['Run End Time'].iloc[0] == incoming_run['Run End Time'].iloc[0])
    metrology_data = global_metrology_data[global_metrology_data['Run ID'] == key]
    print(run['Run Start Time'].iloc[0] == metrology_data['Run Start Time'].iloc[0])
    print(run['Run End Time'].iloc[0] == metrology_data['Run End Time'].iloc[0])
    print("============")

In [24]:
# Define the columns that describe each unique point
time_columns = ['Run ID', 'Run Start Time', 'Run End Time']

# Create a clean DataFrame where each Point Index appears only once
time_mapping_df = test_run_data[time_columns].drop_duplicates()

In [27]:
time_mapping_df['Run ID'] = time_mapping_df['Run ID'].map(run_id_dict)
time_mapping_df

,Run ID,Run Start Time,Run End Time
0,3702,2024-01-01 01:27:55,2024-01-01 01:40:30
10170,953,2024-01-01 04:06:10,2024-01-01 04:18:45
20160,2658,2024-01-01 04:23:45,2024-01-01 04:36:20
29160,56,2024-01-01 07:19:35,2024-01-01 07:32:10
38370,1426,2024-01-01 13:46:25,2024-01-01 13:59:00
...,...,...,...
8506110,1484,2024-01-02 20:19:35,2024-01-02 20:32:10
8517045,1061,2024-01-02 20:37:10,2024-01-02 20:49:45
8527320,3376,2024-01-02 20:54:45,2024-01-02 21:07:20
8537670,4608,2024-01-02 21:12:20,2024-01-02 21:24:55


In [35]:
set(time_mapping_df['Run ID'].unique()) == set(sorted_processed_test_run_data['Run ID'].unique())

False

In [31]:
time_mapping_df['Run ID'].nunique()

860

In [28]:
pred_y_df = pd.merge(
    left=pred_y_df,
    right=time_mapping_df,
    on='Run ID',
    how='left'  # Use a 'left' merge to ensure all rows from sorted_df are kept
)
pred_y_df

,Run ID,Point Index,Measurement,X,Y,X_index,Y_index,Run Start Time,Run End Time
0,3,0,10.204296,3.061224,-119.387755,25,5,NaT,NaT
1,3,40,10.209535,82.653061,-94.897959,38,9,NaT,NaT
2,3,3,10.187499,-143.877551,-9.183673,1,23,NaT,NaT
3,3,46,10.418071,131.632653,9.183673,46,26,NaT,NaT
4,3,9,10.323218,-58.163265,-33.673469,15,19,NaT,NaT
...,...,...,...,...,...,...,...,...,...
42135,4909,13,10.169545,27.551020,-52.040816,29,16,2024-01-03 08:22:45,2024-01-03 08:35:20
42136,4909,33,10.246445,52.040816,88.775510,33,39,2024-01-03 08:22:45,2024-01-03 08:35:20
42137,4909,14,10.062579,70.408163,-131.632653,36,3,2024-01-03 08:22:45,2024-01-03 08:35:20
42138,4909,15,10.273788,76.530612,58.163265,37,34,2024-01-03 08:22:45,2024-01-03 08:35:20


In [33]:
pred_y_df.isnull().sum()

Run ID                0
Point Index           0
Measurement           0
X                     0
Y                     0
X_index               0
Y_index               0
Run Start Time    34398
Run End Time      34398
dtype: int64